In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import DeterministicProcess

In [2]:
# path to the dataset in Kaggle's notebook
path = '../input/store-sales-time-series-forecasting/'

### 1. Compute Moving Average of Oil Prices

In [3]:
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')

########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(7).mean() # change 'None' to your answer


# Create continguous moving average of oil prices
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
calendar = calendar.merge(data_oil, how='left', left_index=True, right_index=True)
calendar['ma_oil'].fillna(method='backfill', inplace=True)

calendar.head(15) # display some entries of calendar

,dcoilwtico,ma_oil
2013-01-01,NaN,93.218571
2013-01-02,93.14,93.218571
2013-01-03,92.97,93.218571
2013-01-04,93.12,93.218571
2013-01-05,NaN,93.218571
2013-01-06,NaN,93.218571
2013-01-07,93.20,93.218571
2013-01-08,93.21,93.218571
2013-01-09,93.08,93.218571
2013-01-10,93.81,93.218571


### 2. Create Workday Feature

In [4]:
########################################################################################################################
# TODO: create a True/False feature calendar['wd'] to indicate whether each date is a workday (Monday-Friday) or not.
# Hint: check documentation of pandas.DatetimeIndex.dayofweek
########################################################################################################################
calendar['wd'] = calendar.index.dayofweek < 5 # change 'None' to your answer
calendar['wd_binary']=calendar['wd'].astype(int)
calendar.head(15) # display some entries of calendar

,dcoilwtico,ma_oil,wd,wd_binary
2013-01-01,NaN,93.218571,True,1
2013-01-02,93.14,93.218571,True,1
2013-01-03,92.97,93.218571,True,1
2013-01-04,93.12,93.218571,True,1
2013-01-05,NaN,93.218571,False,0
2013-01-06,NaN,93.218571,False,0
2013-01-07,93.20,93.218571,True,1
2013-01-08,93.21,93.218571,True,1
2013-01-09,93.08,93.218571,True,1
2013-01-10,93.81,93.218571,True,1


### 3. Read Train and Test Data

In [5]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

df_train.head(15) # display some entries of the training data

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0
                     2013-01-06    2.0
                     2013-01-07    0.0
                     2013-01-08    2.0
                     2013-01-09    2.0
                     2013-01-10    2.0
                     2013-01-11    3.0
                     2013-01-12    2.0
                     2013-01-13    2.0
                     2013-01-14    2.0
                     2013-01-15    1.0

In [6]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

df_test.head(15) # display some entries of the testing data

Empty DataFrame
Columns: []
Index: [(1, AUTOMOTIVE, 2017-08-16), (1, AUTOMOTIVE, 2017-08-17), (1, AUTOMOTIVE, 2017-08-18), (1, AUTOMOTIVE, 2017-08-19), (1, AUTOMOTIVE, 2017-08-20), (1, AUTOMOTIVE, 2017-08-21), (1, AUTOMOTIVE, 2017-08-22), (1, AUTOMOTIVE, 2017-08-23), (1, AUTOMOTIVE, 2017-08-24), (1, AUTOMOTIVE, 2017-08-25), (1, AUTOMOTIVE, 2017-08-26), (1, AUTOMOTIVE, 2017-08-27), (1, AUTOMOTIVE, 2017-08-28), (1, AUTOMOTIVE, 2017-08-29), (1, AUTOMOTIVE, 2017-08-30)]

In [7]:
# set the range of data used in training
sdate = '2017-04-01'
edate = '2017-08-15'

# we will train a model that takes feature of a date as input and predicts the sales for each store and family of goods on that date.
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]


########################################################################################################################
# TODO: create the trend feature X: the value for sdate is 1, the value for the next day of sdate is 2, etc.
# Hint: check the documentation of DeterministicProcess, or this tutorial: https://www.kaggle.com/code/ryanholbrook/trend.
########################################################################################################################
from statsmodels.tsa.deterministic import Fourier

fourier = Fourier(period=7, order=3)
dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          drop=True,additional_terms=[fourier])

X = dp.in_sample() # change 'None' to your answer

# Extentions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values

X.head(15)

,trend,"sin(1,7)","cos(1,7)","sin(2,7)","cos(2,7)","sin(3,7)","cos(3,7)",oil,wd
date,,,,,,,,,
2017-04-01,1.0,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,49.034286,False
2017-04-02,2.0,7.818315e-01,0.623490,9.749279e-01,-0.222521,4.338837e-01,-0.900969,49.034286,False
2017-04-03,3.0,9.749279e-01,-0.222521,-4.338837e-01,-0.900969,-7.818315e-01,0.623490,49.034286,True
2017-04-04,4.0,4.338837e-01,-0.900969,-7.818315e-01,0.623490,9.749279e-01,-0.222521,49.561429,True
2017-04-05,5.0,-4.338837e-01,-0.900969,7.818315e-01,0.623490,-9.749279e-01,-0.222521,50.150000,True
2017-04-06,6.0,-9.749279e-01,-0.222521,4.338837e-01,-0.900969,7.818315e-01,0.623490,50.625714,True
2017-04-07,7.0,-7.818315e-01,0.623490,-9.749279e-01,-0.222521,-4.338837e-01,-0.900969,51.022857,True
2017-04-08,8.0,-2.449294e-16,1.000000,-4.898587e-16,1.000000,-7.347881e-16,1.000000,51.417143,False
2017-04-09,9.0,7.818315e-01,0.623490,9.749279e-01,-0.222521,4.338837e-01,-0.900969,51.417143,False


### 4. Train Model!

In [8]:
model = LinearRegression()
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [9]:
# Results on the training set

y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0

########################################################################################################################
# TODO: show the training loss for each type of product.
# Hint: check the documentation of DataFrame.groupby() and GroupBy.apply().
########################################################################################################################
# y_target.groupby('family').apply(lambda r: mean_squared_error(r['sales'], r['sales_pred']))
y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])) ### answer

family
AUTOMOTIVE                    0.262876
BABY CARE                     0.068387
BEAUTY                        0.269746
BEVERAGES                     0.193554
BOOKS                         0.027975
BREAD/BAKERY                  0.120456
CELEBRATION                   0.294605
CLEANING                      0.210812
DAIRY                         0.132261
DELI                          0.104360
EGGS                          0.140334
FROZEN FOODS                  0.168430
GROCERY I                     0.206394
GROCERY II                    0.353404
HARDWARE                      0.280179
HOME AND KITCHEN I            0.264055
HOME AND KITCHEN II           0.224661
HOME APPLIANCES               0.156139
HOME CARE                     0.119397
LADIESWEAR                    0.262420
LAWN AND GARDEN               0.259770
LINGERIE                      0.406804
LIQUOR,WINE,BEER              0.648438
MAGAZINES                     0.258742
MEATS                         0.114422
PERSONAL CARE     

In [10]:
# Test predictions

stest = '2017-08-16'
etest = '2017-08-31'

########################################################################################################################
# TODO: create the feature matrix of test data.
# Hint: check the documentation of DeterministicProcess.
########################################################################################################################
X_test = dp.out_of_sample(steps=16) # change 'None' to your answer

X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test['wd']   = calendar.loc[stest:etest]['wd'].values

print(X_test.head(10))

sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])

sales_pred[sales_pred < 0] = 0. # Sales should be >= 0

            trend      sin(1,7)  cos(1,7)      sin(2,7)  cos(2,7)  \
2017-08-16  138.0 -4.338837e-01 -0.900969  7.818315e-01  0.623490   
2017-08-17  139.0 -9.749279e-01 -0.222521  4.338837e-01 -0.900969   
2017-08-18  140.0 -7.818315e-01  0.623490 -9.749279e-01 -0.222521   
2017-08-19  141.0 -4.898587e-15  1.000000 -9.797174e-15  1.000000   
2017-08-20  142.0  7.818315e-01  0.623490  9.749279e-01 -0.222521   
2017-08-21  143.0  9.749279e-01 -0.222521 -4.338837e-01 -0.900969   
2017-08-22  144.0  4.338837e-01 -0.900969 -7.818315e-01  0.623490   
2017-08-23  145.0 -4.338837e-01 -0.900969  7.818315e-01  0.623490   
2017-08-24  146.0 -9.749279e-01 -0.222521  4.338837e-01 -0.900969   
2017-08-25  147.0 -7.818315e-01  0.623490 -9.749279e-01 -0.222521   

                sin(3,7)  cos(3,7)        oil     wd  
2017-08-16 -9.749279e-01 -0.222521  48.281429   True  
2017-08-17  7.818315e-01  0.623490  47.995714   True  
2017-08-18 -4.338837e-01 -0.900969  47.852857   True  
2017-08-19 -4.311747

In [11]:
print(sales_pred.head(10))

                                         sales
           store_nbr family                   
2017-08-16 1         AUTOMOTIVE       4.304066
                     BABY CARE        0.000000
                     BEAUTY           4.006828
                     BEVERAGES     2369.555628
                     BOOKS            0.409197
                     BREAD/BAKERY   372.772755
                     CELEBRATION     12.361699
                     CLEANING       802.215281
                     DAIRY          765.680952
                     DELI           135.525270


In [12]:
print(X_test)

            trend      sin(1,7)  cos(1,7)      sin(2,7)  cos(2,7)  \
2017-08-16  138.0 -4.338837e-01 -0.900969  7.818315e-01  0.623490   
2017-08-17  139.0 -9.749279e-01 -0.222521  4.338837e-01 -0.900969   
2017-08-18  140.0 -7.818315e-01  0.623490 -9.749279e-01 -0.222521   
2017-08-19  141.0 -4.898587e-15  1.000000 -9.797174e-15  1.000000   
2017-08-20  142.0  7.818315e-01  0.623490  9.749279e-01 -0.222521   
2017-08-21  143.0  9.749279e-01 -0.222521 -4.338837e-01 -0.900969   
2017-08-22  144.0  4.338837e-01 -0.900969 -7.818315e-01  0.623490   
2017-08-23  145.0 -4.338837e-01 -0.900969  7.818315e-01  0.623490   
2017-08-24  146.0 -9.749279e-01 -0.222521  4.338837e-01 -0.900969   
2017-08-25  147.0 -7.818315e-01  0.623490 -9.749279e-01 -0.222521   
2017-08-26  148.0  1.961911e-15  1.000000  3.923822e-15  1.000000   
2017-08-27  149.0  7.818315e-01  0.623490  9.749279e-01 -0.222521   
2017-08-28  150.0  9.749279e-01 -0.222521 -4.338837e-01 -0.900969   
2017-08-29  151.0  4.338837e-01 -0

In [13]:
# Create submission

df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred.values
df_sub.to_csv('submission.csv', index=True)